In [0]:

# Define the path to the Delta table
delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"
# Read the Delta table to a Spark DataFrame
aws_keys_df = spark.read.format("delta").load(delta_table_path)

In [0]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib

# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [0]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib

# AWS S3 bucket name
AWS_S3_BUCKET = "user-12ffc5aba733-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/12ffc5aba733"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive

try:
  dbutils.fs.unmount(SOURCE_URL, MOUNT_NAME)
except:
  pass

try:
  dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)
except:
  pass

In [0]:
%sql
SET spark.databricks.delta.formatCheck.enabled=false

In [0]:

dbutils.fs.ls('/mnt/12ffc5aba733/topics/12ffc5aba733.pin/partition=0/')

In [0]:
# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location = "/mnt/12ffc5aba733/topics/12ffc5aba733.pin/partition=0/*.json" 
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
df_pin = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)
# Display Spark dataframe to check its content
display(df_pin)

In [0]:
# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location = "/mnt/12ffc5aba733/topics/12ffc5aba733.geo/partition=0/*.json" 
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
df_geo = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)
# Display Spark dataframe to check its content
display(df_geo)

In [0]:
# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location = "/mnt/12ffc5aba733/topics/12ffc5aba733.user/partition=0/*.json" 
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
df_user = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)
# Display Spark dataframe to check its content
display(df_user)

In [0]:
loaded_df = spark.table("df_geo")
display(loaded_df)

In [0]:
display(df_pin)

In [0]:

from pyspark.sql.functions import *
from pyspark.sql.types import *

# Replace empty entries or invalid data with None
df_pin_cleaned = df_pin.replace(["", "N/A","User Info Error", "No description available Story format", "Image src error","N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e", "No Title Data Available",], None)
# Clean and cast `follower_count` to IntegerType
df_pin_cleaned = df_pin_cleaned.withColumn("follower_count", regexp_replace(col("follower_count"), " ", ""))
df_pin_cleaned = df_pin_cleaned.withColumn("follower_count",
    when(col("follower_count").contains("k"), 
         regexp_replace(col("follower_count"), "k", "").cast("float") * 1000)
    .when(col("follower_count").contains("m"), 
         regexp_replace(col("follower_count"), "m", "").cast("float") * 1000000)
    .when(col("follower_count").contains("M"), 
         regexp_replace(col("follower_count"), "M", "").cast("float") * 1000000)
    .otherwise(col("follower_count").cast("float"))
)

df_pin_cleaned = df_pin_cleaned.withColumn("follower_count", col("follower_count").cast("int"))

#General cleaning on other columns

df_pin_cleaned = df_pin_cleaned.withColumn("downloaded", col("downloaded").cast(IntegerType()))
df_pin_cleaned = df_pin_cleaned.withColumn("index", col("index").cast(IntegerType()))

df_pin_cleaned = df_pin_cleaned.withColumn("save_location", split(col("save_location"), " ").getItem(3))

#Rename columns
                
df_pin_cleaned = df_pin_cleaned.withColumnRenamed("index", "ind")

# Reorder the DataFrame columns 

df_pin_cleaned = df_pin_cleaned.select("ind", "unique_id", "title", "description", 
                                       "follower_count", "poster_name", "tag_list", 
                                       "is_image_or_video", "image_src", "save_location", "category")
display(df_pin_cleaned)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Create an array between lat and long

df_geo_cleaned = df_geo.withColumn("coordinates", array(col("latitude"), col("longitude")))

# Drop columns from the DataFrame.
df_geo_cleaned = df_geo_cleaned.drop("latitude").drop("longitude")

# Convert the timestamp column from a string to a timestamp data type.
df_geo_cleaned = df_geo_cleaned.withColumn("timestamp", col("timestamp").cast(TimestampType()))

# Reorder the DataFrame columns 
df_geo_cleaned = df_geo_cleaned.select("ind", "country", "coordinates", "timestamp")

display(df_geo_cleaned)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

#Concat both first name and last name via a space. 

df_user_cleaned = df_user.withColumn("user_name", concat_ws(" ", col("first_name"), col("last_name")))

# Drop the columns from the DataFrame.
df_user_cleaned = df_user_cleaned.drop("first_name").drop("last_name")

# Convert the column from a string to a timestamp data type.
df_user_cleaned = df_user_cleaned.withColumn("date_joined", col("date_joined").cast(TimestampType()))

#  Reorder the DataFrame columns 
df_user_cleaned = df_user_cleaned.select("ind", "user_name", "age", "date_joined")

display(df_user_cleaned)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

#Task 4

df_combined = df_pin_cleaned.join(df_geo_cleaned, df_geo_cleaned["ind"] == df_pin_cleaned["ind"], how="inner")

#Group by country and category, and count the occurrences of each category in each country

category_count_df = df_combined.groupBy("country", "category").agg(count("*").alias("category_count"))

sorted_df = category_count_df.orderBy("category_count", ascending=False)

display(sorted_df )


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

#Task 5 

#Join on both df

df_combined_2 = df_pin_cleaned.join(df_geo_cleaned, on="ind")

#Group by timestamp and catergory where there is matching

category_count_df_2 = df_combined_2.groupBy(year("timestamp").alias("post_year"), "category") \
                               .agg(count("*").alias("category_count"))

# Filter years between 2018 and 2022
filtered_df_2 = category_count_df_2.where(col("post_year").between(2018, 2022))

# Sort the DataFrame by category_count in descending order
sorted_df_2 = filtered_df_2.orderBy("category_count", ascending=False)

display(sorted_df_2 )

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

#Task 6

df_combined_3 = df_pin_cleaned.join(df_geo_cleaned, df_geo_cleaned["ind"] == df_pin_cleaned["ind"], how="inner")

df_country_max_followers = df_combined_3.groupBy("country", "poster_name") \
            .agg(max("follower_count").alias("follower_count"))


sorted_df_3 = df_country_max_followers.orderBy("follower_count", ascending=False)


#display(sorted_df_3)


#part 2

df_max_follower_country = df_country_max_followers.orderBy(col("follower_count").desc()).limit(1).select("country", "follower_count")

display(df_max_follower_country)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

#Task 7
df_combined_4 = df_pin_cleaned.join(df_user_cleaned, df_user_cleaned["ind"] == df_pin_cleaned["ind"], how="inner")

df_age_grouped = df_combined_4.withColumn(
    "age_group",
    when(col("age").between(18, 24), "18-24")
    .when(col("age").between(25, 35), "25-35")
    .when(col("age").between(36, 50), "36-50")
    .when(col("age") > 50, "50+")
)


df_ages = df_age_grouped.groupBy("age_group", "category") \
            .agg(count("*").alias("category_count"))

sorted_df_4 = df_ages.orderBy("category_count", ascending=False)

display(sorted_df_4 )

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

#Task 8
df_combined_5 = df_pin_cleaned.join(df_user_cleaned, df_user_cleaned["ind"] == df_pin_cleaned["ind"], how="inner")

df_age_grouped = df_combined_5.withColumn(
    "age_group",
    when(col("age").between(18, 24), "18-24")
    .when(col("age").between(25, 35), "25-35")
    .when(col("age").between(36, 50), "36-50")
    .when(col("age") > 50, "50+")
)

df_ages_2 = df_age_grouped.groupBy("age_group") \
            .agg(percentile_approx("follower_count", 0.5).alias("median_follower_count"))

sorted_df_5 = df_ages_2.orderBy("median_follower_count", ascending=False)            


display(sorted_df_5)            


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

#Task 9


df_age_grouped_2 = df_user_cleaned.withColumn(
    "year_joined",
    when(year("date_joined") == 2015, "2015")
    .when(year("date_joined") == 2016, "2016")
    .when(year("date_joined") == 2017, "2017")
    .when(year("date_joined") == 2018, "2018")
    .when(year("date_joined") == 2019, "2019")
    .when(year("date_joined") == 2020, "2020")
    

)



users_joined_df = df_age_grouped_2.groupBy(year("year_joined").alias("post_year")) \
                               .agg(count("*").alias("number_users_joined"))


sorted_df_6 = users_joined_df.orderBy("number_users_joined", ascending=False)         


display(sorted_df_6 )

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

df_with_year = df_user_cleaned.withColumn("post_year", year("date_joined"))

df_filtered = df_with_year.filter(col("post_year").between(2015, 2020))

users_joined_df = df_filtered.groupBy("post_year").agg(count("*").alias("number_users_joined"))

sorted_df = users_joined_df.orderBy("post_year", ascending=True)

sorted_df.show()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

#task 10

df_combined_6 = df_pin_cleaned.join(df_user_cleaned, df_user_cleaned["ind"] == df_pin_cleaned["ind"], how="inner")

df_with_year = df_combined_6.withColumn("post_year", year("date_joined"))

df_filtered = df_with_year.filter(col("post_year").between(2015, 2020))

users_joined_df = df_filtered.groupBy("post_year").agg(percentile_approx("follower_count", 0.5).alias("median_follower_count"))

sorted_df_7 = users_joined_df.orderBy("median_follower_count", ascending=False)    

sorted_df_7.show()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

#task 11

df_combined_6 = df_pin_cleaned.join(df_user_cleaned, df_user_cleaned["ind"] == df_pin_cleaned["ind"], how="inner")

df_with_year = df_combined_6.withColumn("post_year", year("date_joined"))

df_filtered = df_with_year.filter(col("post_year").between(2015, 2020))

df_age_year = df_filtered.withColumn(
    "age_group",
    when(col("age").between(18, 24), "18-24")
    .when(col("age").between(25, 35), "25-35")
    .when(col("age").between(36, 50), "36-50")
    .when(col("age") > 50, "50+")
)

df_grouped_final = df_age_year.groupBy("post_year", "age_group").agg(percentile_approx("follower_count", 0.5).alias("median_follower_count"))

sorted_df_8 = df_grouped_final.orderBy("post_year", ascending=False)    

sorted_df_8.show()